In [ ]:
import os
from os import path
from astropy.time import Time
from astropy.io import fits, ascii
import astropy.units as u
from astropy.table import Table
from astropy.constants import G

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from matplotlib.gridspec import GridSpec
import h5py
import schwimmbad

from thejoker import JokerSamples
from thejoker.sampler import JokerParams, TheJoker
from thejoker.plot import plot_rv_curves

from twoface.config import TWOFACE_CACHE_PATH
from twoface.db import (db_connect, AllStar, AllVisit, AllVisitToAllStar, RedClump,
                        StarResult, Status, JokerRun, initialize_db)
from twoface.data import APOGEERVData
from twoface.plot import plot_data_orbits
from twoface.samples_analysis import unimodal_P

In [ ]:
samples_file = path.join(TWOFACE_CACHE_PATH, 'apogee-jitter.hdf5')

In [ ]:
Session, _ = db_connect(path.join(TWOFACE_CACHE_PATH, 'apogee.sqlite'))
session = Session()

In [ ]:
run = session.query(JokerRun).filter(JokerRun.name == 'apogee-jitter').one()
params = run.get_joker_params()

In [ ]:
# needs mcmc
stars = session.query(AllStar).join(StarResult, JokerRun, Status)\
               .filter(JokerRun.name == 'apogee-jitter')\
               .filter(Status.id == 2).all()
len(stars)

In [ ]:
star = stars[0]

data = star.apogeervdata()
with h5py.File(samples_file) as f:
    samples0 = JokerSamples.from_hdf5(f[star.apogee_id])

_ = plot_data_orbits(data, samples0, xlim_choice='tight')

In [ ]:
%%time

with schwimmbad.MultiPool() as pool:
    joker = TheJoker(params, pool=pool)
    model, samples, sampler = joker.mcmc_sample(data, samples0, n_steps=32768,
                                                n_walkers=256, n_burn=1024,
                                                return_sampler=True)

In [ ]:
ndim = sampler.chain.shape[-1]

fig, axes = plt.subplots(ndim, 2, figsize=(10, 16))
for k in range(ndim):
    for walker in sampler.chain[..., k]:
        axes[k, 0].plot(walker, marker='', drawstyle='steps-mid', alpha=0.1)
        
    axes[k, 1].plot(np.std(sampler.chain[..., k], axis=0),
                    marker='', drawstyle='steps-mid')

In [ ]:
_ = plot_data_orbits(data, samples0, xlim_choice='tight')
_ = plot_data_orbits(data, samples, xlim_choice='tight', highlight_P_extrema=False)

In [ ]:
plt.scatter(samples['P'].value, samples['e'].value, alpha=0.5, linewidth=0)

In [ ]:
for star in stars2:
    with h5py.File(samples_file) as f:
        samples = JokerSamples.from_hdf5(f[star.apogee_id])

    # compute minimum companion mass
    if star.martig_filter:
        m2_mins = []
        for i in range(len(samples)):
            orbit = samples.get_orbit(i)
            res = root(m2_func, x0=10, args=(star.martig_mass, 1., orbit.m_f.value))
            m2_mins.append(res.x[0])
        m2_mins = m2_mins*u.Msun
        # print('m1 = {0:.3f}'.format(star.martig_mass*u.Msun))
        # print('m2 = {0:.3f}, {1:.2f}'.format(np.mean(m2_mins), np.mean(m2_mins).to(u.Mjup)))
        m1 = star.martig_mass*u.Msun
        m2 = np.mean(m2_mins)

    else:
        m1 = None
        m2 = None

    # acceleration percentile
    acc = acc_per[all_apogee_ids == star.apogee_id][0] * K_unit/P_unit

    fig = make_two_panel(star, m1=m1, m2=m2, acc=acc)
    fig.savefig('../plots/needs-mcmc/{0}-2panel.png'.format(star.apogee_id), dpi=200)
    
    fig = make_phase_fold(star)
    fig.savefig('../plots/needs-mcmc/{0}-residual.png'.format(star.apogee_id), dpi=200)
    
    plt.close('all')